In [1]:
import pandas as pd

In [2]:
from pycparser import c_parser, c_ast
import pandas as pd
import os
import re
import sys
from gensim.models.word2vec import Word2Vec
import pickle
from tree import ASTNode, SingleNode
import numpy as np


def get_sequences(node, sequence):
    current = SingleNode(node)
    sequence.append(current.get_token())
    for _, child in node.children():
        get_sequences(child, sequence)
    if current.get_token().lower() == 'compound':
        sequence.append('End')


def get_blocks(node, block_seq):
    children = node.children()
    name = node.__class__.__name__
    if name in ['FuncDef', 'If', 'For', 'While', 'DoWhile']:
        block_seq.append(ASTNode(node))
        if name is not 'For':
            skip = 1
        else:
            skip = len(children) - 1

        for i in range(skip, len(children)):
            child = children[i][1]
            if child.__class__.__name__ not in ['FuncDef', 'If', 'For', 'While', 'DoWhile', 'Compound']:
                block_seq.append(ASTNode(child))
            get_blocks(child, block_seq)
    elif name is 'Compound':
        block_seq.append(ASTNode(name))
        for _, child in node.children():
            if child.__class__.__name__ not in ['If', 'For', 'While', 'DoWhile']:
                block_seq.append(ASTNode(child))
            get_blocks(child, block_seq)
        block_seq.append(ASTNode('End'))
    else:
        for _, child in node.children():
            get_blocks(child, block_seq)


In [3]:
blocks = pd.read_pickle('blocks.pkl')

In [4]:
tmp = list(blocks['code'])[0]

In [5]:
tmp

[[32, [2, [30, [40, [81]]]]],
 [6],
 [2, [9, [4]]],
 [2, [565, [4]]],
 [2, [1105, [4]]],
 [2, [2554, [4]]],
 [11, [36], [12, [46], [29, [9]]]],
 [3, [565], [5]],
 [3, [1105], [5]],
 [3, [2554], [5]],
 [15, [19, [53, [9], [70]], [5]]],
 [3, [565], [10]],
 [15, [19, [53, [9], [67]], [5]]],
 [3, [1105], [10]],
 [15, [19, [53, [9], [95]], [5]]],
 [3, [2554], [10]],
 [15, [19, [565], [10]]],
 [6],
 [11, [25], [12, [597]]],
 [15, [19, [1105], [10]]],
 [6],
 [11, [25], [12, [1916]]],
 [15, [19, [2554], [10]]],
 [11, [25], [12, [1547]]],
 [7],
 [15, [19, [2554], [10]]],
 [11, [25], [12, [1547]]],
 [7],
 [15, [1105]],
 [6],
 [11, [25], [12, [595]]],
 [15, [2554]],
 [11, [25], [12, [1547]]],
 [7],
 [15, [2554]],
 [6],
 [11, [25], [12, [585]]],
 [7],
 [11, [25], [12, [562]]],
 [7]]

In [19]:
[1][1:]

[]

In [8]:
import dgl
g = dgl.DGLGraph()
    

In [16]:
g.add_nodes(1)
print(g.number_of_nodes())
g.add_nodes(1)
print(g.number_of_nodes())


7
8


In [18]:
type([11]) == type([])

True

In [10]:
from gensim.models.word2vec import Word2Vec

In [12]:
word2vec = Word2Vec.load('embedding/node_w2v_128').wv

In [32]:
word2vec.vocab

{'FileAST': <gensim.models.keyedvectors.Vocab at 0x1df409635c0>,
 'FuncDef': <gensim.models.keyedvectors.Vocab at 0x1df409636a0>,
 'Decl': <gensim.models.keyedvectors.Vocab at 0x1df40963710>,
 'FuncDecl': <gensim.models.keyedvectors.Vocab at 0x1df40963748>,
 'main': <gensim.models.keyedvectors.Vocab at 0x1df409637b8>,
 'void': <gensim.models.keyedvectors.Vocab at 0x1df40963828>,
 'Compound': <gensim.models.keyedvectors.Vocab at 0x1df40963860>,
 'a': <gensim.models.keyedvectors.Vocab at 0x1df40963898>,
 'int': <gensim.models.keyedvectors.Vocab at 0x1df40963908>,
 'i3': <gensim.models.keyedvectors.Vocab at 0x1df40963978>,
 'i5': <gensim.models.keyedvectors.Vocab at 0x1df409639e8>,
 'i7': <gensim.models.keyedvectors.Vocab at 0x1df40963a58>,
 'FuncCall': <gensim.models.keyedvectors.Vocab at 0x1df40963a90>,
 'scanf': <gensim.models.keyedvectors.Vocab at 0x1df40963b00>,
 'ExprList': <gensim.models.keyedvectors.Vocab at 0x1df40963b38>,
 '"%d"': <gensim.models.keyedvectors.Vocab at 0x1df40963b

In [17]:
tmp_rst = []
def list2word(l):
    if type(l) == type([]):
        for e in l:
            list2word(l)
    else:
        l = word2vec.index2word[l]
list2word(tmp)

RecursionError: maximum recursion depth exceeded while calling a Python object

In [20]:
train_pkl = pd.read_pickle('train_.pkl')

In [21]:
train_pkl

,id,code,label
46887,46887,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",93
4970,4970,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",42
50521,50521,"FileAST(ext=[Decl(name='i',\n ...",77
38406,38406,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",63
44230,44230,"FileAST(ext=[Decl(name='DiJiTian',\n ...",80
21122,21122,"FileAST(ext=[FuncDef(decl=Decl(name='f',\n ...",69
31463,31463,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",84
43363,43363,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",88
16692,16692,"FileAST(ext=[FuncDef(decl=Decl(name='main',\n ...",7
9937,9937,"FileAST(ext=[FuncDef(decl=Decl(name='get',\n ...",8


In [22]:
train_pkl0 = list(train_pkl['code'])[0]

In [23]:
train_pkl0

FileAST(ext=[FuncDef(decl=Decl(name='main',
                               quals=[
                                     ],
                               storage=[
                                       ],
                               funcspec=[
                                        ],
                               type=FuncDecl(args=None,
                                             type=TypeDecl(declname='main',
                                                           quals=[
                                                                 ],
                                                           type=IdentifierType(names=['void'
                                                                                     ]
                                                                               )
                                                           )
                                             ),
                               init=None,
                              

In [26]:
block_tmp = []
train_pkl0_block = get_blocks(train_pkl0,block_tmp)

In [30]:
len(block_tmp)

40

In [31]:
train_pkl0.__class__.__name__

'FileAST'